##Lab 4. Text generation with deep learning
Melnikov, Malysheva, Selivanovskaya




In [1]:
from collections import Counter
import csv

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

###Считывание данных из файла


In [2]:
'''TRAIN_TEXT_FILE_PATH = 'train_text.txt'

with open(TRAIN_TEXT_FILE_PATH, 'r', encoding='cp1251') as text_file:
    text_sample = text_file.read()'''


"TRAIN_TEXT_FILE_PATH = 'train_text.txt'\n\nwith open(TRAIN_TEXT_FILE_PATH, 'r', encoding='cp1251') as text_file:\n    text_sample = text_file.read()"

In [3]:
def DataLoader(filename): #function to load arxiv.csv
    corpus = list()

    with open(filename, encoding='utf-8') as r_file:
        file_reader = csv.reader(r_file, delimiter = ",")
        count = 0
        for row in file_reader:
            if count != 0:
                line = row[7]
                line = line.replace('\n', ' ')
                corpus.append(line)
            count += 1
    return corpus


def text_to_seq(text_sample): #making dicts with words and indexes (char_to_idx, idx_to_char)
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

In [5]:
! git clone https://github.com/edmelnikov/Computational-linguistics-course-lab4

Cloning into 'Computational-linguistics-course-lab4'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [7]:
corpus_list = DataLoader('Computational-linguistics-course-lab4/arxiv.csv') 
train_arxiv = ' '.join(corpus_list)

In [8]:
sequence, char_to_idx, idx_to_char = text_to_seq(train_arxiv)

[' ', 'e', 't', 'a', 'i', 'o', 'n', 's', 'r', 'l', 'c', 'h', 'd', 'm', 'p', 'u', 'f', 'g', 'y', 'b', 'w', 'v', '.', ',', '-', 'k', 'x', 'T', '$', ')', '(', 'I', 'q', 'S', 'W', 'z', 'A', 'C', 'M', 'N', 'P', 'D', 'R', '\\', 'O', 'F', 'L', 'E', 'j', 'B', '1', 'G', '2', 'H', '0', '}', '{', 'U', 'V', "'", '3', ':', '"', '/', '_', '5', 'K', '^', '4', '6', '%', '9', 'Q', '8', '7', ';', '+', 'X', '=', 'J', '[', ']', 'Y', 'Z', '|', '?', '*', '>', '~', '<', '`', '&', '!', '#', '@', '\x7f']


###Генерация батчей для обучения текста

In [13]:
BATCH_SIZE = 16
SEQ_LENGTH = 256

def batch_generation(sequence):
    targets = []
    trains = []
    for _ in range(BATCH_SIZE):
        start_batch = np.random.randint(0, len(sequence) - SEQ_LENGTH)
        chunk = sequence[start_batch: start_batch + SEQ_LENGTH]
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        targets.append(target)
        trains.append(train)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

###Функция генерирующая текст 
Удобно использовать при обучении

In [15]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', pred_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    pred_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(pred_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_ind = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_ind]).view(-1, 1, 1).to(device)
        pred_char = idx_to_char[top_ind]
        pred_text += pred_char
    
    return pred_text

###Класс модели

In [16]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

###Обучение

В качестве функции потерь мы используем CrossEntropyLoss()

Сначала обучим модель на корпусе из англоязычных текстов:

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = RNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)

#model.load_state_dict(torch.load(TRAIN_MODEL_FILE_PATH)) 
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = batch_generation(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.8356025552749635
 umender the the the the the ant and an and the the ant ant ant ang the the the the the the bolate sale the an the an potion ant on sentrent on the the the and the thar on the the the the ant the deren
Loss: 2.214913830757141
 propering constression stration the the the the proces the is constrition the dentrith a properentic the and the and and the sulled provers the the the and sedic the the in the parition as preman the 
Loss: 1.98653386592865
 strations and and the and the and over the the the trames the constrition the to can to constrighes and of the sing stration station for the provel and and and and and the model the stration and the c
Loss: 1.844714183807373
 be propose a the process the sent and state the propose and the sures the propose of the problem and model spection of the state the contrated the proposed and the result the proposed the propose the 
Loss: 1.759784963130951
 spectrive the results the approach and the with the and the seconsing a

KeyboardInterrupt: ignored

Как видно по результатам обучения, вначале модель выдает несвязные последовательности словосочетаний:

Loss: 2.8356025552749635


umender the the the the the ant and an and the the ant ant ant ang the the the the the the bolate sale the an the an potion ant on sentrent on the the the and the thar on the the the the ant the deren





###Сохранение модели 

In [ ]:
torch.save(model.state_dict(), 'model_trained_arxiv.pt')
model = RNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)

In [19]:
model.load_state_dict(torch.load('Computational-linguistics-course-lab4/model_trained_arxiv.pt')) 

<All keys matched successfully>

###Гененирование текста на обученной модели 

In [21]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.5, 
    pred_len=1000, 
    start_text='. '
    )
)

.  We also propose a new increases on a given measurements which is an information algorithm to the structure of the recurrent factors energy in this paper we sensing how the information of the produce the baseline of a set of the complexity and the theory of the model to the set of series in the baseline in different scales, and a set of the conventional interference and the algorithm to perform the construction for these state to the source different process. We also propose a novel synthesis between the comparisons in constraints and selection of the problem show that the problem of previous algorithm to the state-of-the-art prediction is the same distribution is a state of the computational graph to a set of algorithm is of the same networks for automatically the computational methods for the analysis of our results and state-of-the-art methods to developed to search to be a detection and studied for a computational policy setting study the statistical structure for the proposed pr

После обучения генерируемые последовательности словосочетаний все равно не имеют смысла. Однако, отдельные словосочетания и расстановка предлогов представляются более осмысленными. 

###Обучим модель на корпусе из русскоязычных текстов

Для обучения будем использовать стихи Владимира Маяковского.

In [31]:
with open('Computational-linguistics-course-lab4/all_poems.txt') as inp:
    poems=inp.read()
sequence, char_to_idx, idx_to_char = text_to_seq(poems)

[' ', 'о', 'и', 'а', 'е', 'т', 'н', 'р', 'с', 'л', 'в', 'к', 'д', 'м', 'у', 'п', ',', 'ь', '.', 'я', 'ы', 'б', 'з', 'г', '̆', 'ч', 'ж', 'ш', 'х', '—', 'ю', '!', 'ц', 'щ', 'В', 'П', 'Н', 'С', 'К', ':', 'э', '?', 'И', '-', 'ф', 'М', 'А', '«', '»', 'Т', 'О', 'Р', 'Д', 'Ч', 'Б', '́', 'Г', 'З', 'Л', 'Е', '̈', 'У', 'Э', 'Я', 'Х', 'Ф', 'ъ', '(', ')', 'Ш', 'Ж', '*', ';', '̀', 'Ц', 'I', 'e', 'Ю', 'Ь', '[', ']', 'a', 'o', 'r', 'Щ', '·', 'i', '–', 'n', 'l', 'd', 'u', '%', 't', 'Ы', 's', 'm', 'c', '"', 'S', 'h', '\u200e', '’', 'N', '⁄', 'A', 'V', 'C', 'D', 'G', 'O', 'P', 'g', 'b', 'T', 'ѣ', 'p', 'v', 'E', '/', '>', 'X', 'y', 'R', '<', 'U', 'K', 'w', 'B', 'H', '_', 'f', '×', '=', 'L', 'x', 'J', 'j', 'z', 'Z', 'F', 'M']


In [37]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = RNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)

#model.load_state_dict(torch.load(TRAIN_MODEL_FILE_PATH)) 
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 10000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = batch_generation(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 3.188735256195068
 на педи сне вел попотет же не болен посна с вем потоз си поне сибит вот на с роноре горе в попои вано рев потани с повози позтота засне поте с повом сит не вет рени пито в нои порона сто ве сна поне п
Loss: 2.6968464183807375
 моста сто зерить покото сто не ракото бодоли сто отольна на постаресть порестай за в стать не сто вороз стольте пролам в озет не сеста сто в вошит на ратом насто в стое на стостали потость за на и в 
Loss: 2.532141375541687
 сто в на подали ни посторовов продить стовововали поставто в стовол отом сропесть постаков сто костовики на стововово стовет под на позверта — полся сето сторого побостов побочит на в стольстевет слог
Loss: 2.4457282257080077
 в дововов подотов подарить сторить порятить от в подом берить стали и столи в работ на поворать слова корона и свой сабовает доли и строворить сам в подавовали мольсти в и разна на в на посестьят на 
Loss: 2.377684245109558
 постально стуриние на не не дели. Урать подоли и буденной подудина н

In [40]:
torch.save(model.state_dict(), 'trained_model2.pt')

###Сгенерируем текст на обученной модели

In [41]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.5, 
    pred_len=1000, 
    start_text='. '
    )
)

. лозунг с крестьянин и станик на стороны таким. На приказом и свой в нас не трудовой рабочий помощь не под точу не будет которые полена — и слова лето. В паровозов в когда на слезать на дальше — семью обыватель с город собрали в тем богатый не выскоченный стекло молодом и на труда лет. На страницы под дерево и то в берегами с нас и подмести не снова по столетия было страх за клич, и в борьбы и шагав — назад в картен и слез и в небе прошел весело на голодных светку в карман. И вот есть и поле товарищ стройка — и красное и приставай не помощь руку голодный раздутанов на деревне по править братить кормите — так стоно со всем армия в восторга. На свои из рабочий дома стоит и скажет, разбить в того слово протеренной не солнце с стенки — не было в под которого растраты. Не под шар не похожи простить и восторговно поднял стройской простой полезной старите так и сторобы и лентах не заведут вам бы и под нами облеград. Больше за управить наторов в страсти! Как он на солнце вставал

Модель, обученная на русскоязычных текстах имеет больший loss. В целом, с помощью нее так и не удалось получить более-менее осмысленные последовательности словосочетаний.